### I am going to use a file which contains all the zip code pairs that are witin 50 miles of each other. You can download this file from the National Bureau of Economic Research's ZIP Code Distance Database: http://data.nber.org/data/zip-code-distance-database.html

In [4]:
import pandas as pd

zcta5 = pd.read_csv('gaz2016zcta5distance50miles.csv')

# Alternatively, you can directly read the csv file from their website
# url = "http://data.nber.org/distance/2016/gaz/zcta5/gaz2016zcta5distance50miles.csv"
# zcta5 = pd.read_csv(url)

print(zcta5.shape)
zcta5.tail()

(5853510, 3)


,zip1,zip2,mi_to_zcta5
5853505,99927,99830,43.954377
5853506,99927,99836,45.909062
5853507,99927,99833,49.214845
5853508,99929,99903,40.772611
5853509,99929,99918,49.714708


### As shown above, the file contains 5,853,510 pairs of zip codes which are within 50 miles of each other. The last column mi_to_zcta5 is the distance (in miles) between the two zip codes. 

In [2]:
# Load the file which contains your store ids and the zip codes associated with the stores
store_zip = pd.read_excel('store_zip.xlsx')
print(store_zip.shape)
store_zip.head()

(14, 2)


,Zip,Store #
0,94550,910
1,85641,2809
2,22102,1920
3,22302,3250
4,22015,4002


### In this example spreadsheet we just loaded, we have 14 stores and their associated home store zip codes.

In [3]:
# Add a column to store the distance between a zip code and the store.
# Set the initial value for these existing 14 records to 0, 
# since the closest store for each zip code here is by default the listed home store 
store_zip['dist_to_store'] = 0
store_zip.rename(columns={'Zip': 'zip', 'Store #': 'closest_store'}, inplace=True)

# Make a dictionary in order to map zip code to their closest store later
di = pd.Series(store_zip['closest_store'].values, index=store_zip['zip']).to_dict()

store_zip.head()

,zip,closest_store,dist_to_store
0,94550,910,0
1,85641,2809,0
2,22102,1920,0
3,22302,3250,0
4,22015,4002,0


In [5]:
# Every zip code within 50 mile radius of our 14 stores' zip codes
store_ziplist = store_zip['zip'].tolist()

# Find all the zip codes associated with the 14 home store zip codes in the zcta5 table
# These are the zip codes within 50 miles of our 14 stores
zip_within50miles = zcta5[zcta5['zip1'].isin(store_ziplist)==True]
zip_within50miles_list = zip_within50miles['zip2'].unique().tolist()

print("There are %d zip codes within our %d stores' 50 mile radius." % (len(zip_within50miles_list),len(store_ziplist)))

There are 1179 zip codes within our 14 stores' 50 mile radius.


In [6]:
# Create three empty lists to store the results from the iteration
result_zip = []
result_store = []
result_dist = []

# Iterate through each zip code within 50 mile radius and find their closest stores
for i in range(len(zip_within50miles_list)):
    zipcode = zip_within50miles_list[i]
    # A subset for this zip code
    zip_dist = zip_within50miles[zip_within50miles['zip2']==zipcode]
    # The index for the minimum distance in this subset
    min_dist_ix = zip_dist['mi_to_zcta5'].idxmin()
    # Find the closest zip code based on the min index
    closest_zip = zip_dist['zip1'][min_dist_ix]
    # Find the minimum distance based on the min index
    min_dist = zip_dist['mi_to_zcta5'][min_dist_ix]
    # Use the mapping table we created before to link the zip code to the store
    closest_store = di[closest_zip]
    # Store the results in the empty lists
    result_zip.append(zipcode)
    result_store.append(closest_store)
    result_dist.append(min_dist)

# Put the results in a dataframe
result = {'zip': result_zip,
          'closest_store': result_store,
          'dist_to_store': result_dist}

df = pd.DataFrame.from_dict(result)

df = df[['zip', 'closest_store', 'dist_to_store']] 

print(df.shape)
df.head()

(1179, 3)


,zip,closest_store,dist_to_store
0,20721,1804,3.846953
1,20715,1804,4.574063
2,20774,1804,4.793460
3,21035,1804,4.823621
4,20720,1804,5.443201


## We found 1,179 zip codes within 50 miles of the 14 stores. For each zip code, we found the closest store and the distance between the zip code and the store.

In [7]:
# Let's clean the result a little bit 
# Exclude the zip codes associated with a home store - their closest stores should be those home stores by default
df = df[~df['zip'].isin(store_ziplist)]

# Append these home store zip codes and their associated stores to the data frame
result = df.append(store_zip) 

In [8]:
result.shape

(1182, 3)

In [9]:
result.to_csv('Zip Codes Within 50 Miles of Your Store.csv',index=False)